# 원 검출

In [ ]:
import matplotlib.pyplot as plt

train_path = "" # 이미지 데이터 위치 넣기
# load_images_and_contours -> opencv로 구성했었음(보안상의 이유로 삭제)
train_images, train_contours = load_images_and_contours(train_path)

# 첫 번째 이미지와 컨투어 시각화
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Resized Image")
plt.imshow(train_images[1].squeeze(), cmap='gray')

plt.subplot(1, 2, 2)
plt.title("Contour Image")
plt.imshow(train_contours[1].squeeze(), cmap='gray')

plt.show()


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# 이미지 데이터를 읽고 contour를 추출하는 함수
def load_images_and_contours(train_path):
    images = []
    contours = []

    for filename in os.listdir(train_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # 이미지 파일 필터
            img_path = os.path.join(train_path, filename)
            
            # 이미지 불러오기
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image_resized = cv2.resize(image, (128, 128))  # 필요한 크기로 리사이징
            images.append(image_resized)

            # 이진화 및 contour 추출
            _, thresholded = cv2.threshold(image_resized, 220, 255, cv2.THRESH_BINARY)
            contour_img = np.zeros_like(image_resized)
            contours_detected, _ = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(contour_img, contours_detected, -1, (255, 255, 255), 1)
            contours.append(contour_img)

    return np.array(images), np.array(contours)

train_path = ""

# 이미지 및 컨투어 데이터 불러오기
train_images, train_contours = load_images_and_contours(train_path)

train_images = train_images / 255.0
train_contours = train_contours / 255.0


# 데이터 차원 확장 (CNN 입력 형식에 맞게)
train_images = train_images[..., np.newaxis]  # (num_samples, height, width, 1)
train_contours = train_contours[..., np.newaxis]

# 데이터를 train/validation 세트로 분할
train_images, val_images, train_contours, val_contours = train_test_split(
    train_images, train_contours, test_size=0.2, random_state=42
)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # 다운샘플링
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # 병목 (Bottleneck)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # 업샘플링
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# 모델 생성
input_shape = (128, 128, 1)  # 흑백 이미지의 경우
model = unet_model(input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_images, train_contours, epochs=100, batch_size=2, validation_data=(val_images, val_contours))

# 훈련 및 검증 손실 시각화
plt.figure(figsize=(12, 4))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()



In [ ]:
# 이미지 불러오기
test_image_path = ""

test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
test_image = cv2.resize(test_image, (128,128))

_, test_threshold = cv2.threshold(test_image, 220, 255, cv2.THRESH_BINARY)

test_image_resized = test_threshold


# 모델 입력에 맞게 차원 추가
test_image_resized = test_image_resized[..., np.newaxis]
test_image_resized = np.expand_dims(test_image_resized, axis=0)  # 배치 차원 추가


In [ ]:
# UNET 인공지능 활용
result = model.predict(test_image_resized)

plt.imshow(result[0,:,:,0],cmap='gray')
plt.axis('off')

In [ ]:
result[0, :, :, 0]

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


# 흑백 이미지를 3채널 이미지로 변환
original_image = cv2.cvtColor(test_image, cv2.COLOR_GRAY2BGR)
original_image = cv2.resize(original_image, (128, 128))

# UNET 모델 예측 결과를 이진 마스크로 변환
mask = (result[0, :, :, 0] > 0.1).astype(np.uint8)  # 0.5를 기준으로 이진화

# 빨간색 채널 생성 (빨간색으로 표시)
red_overlay = np.zeros_like(original_image)
red_overlay[:, :, 2] = mask * 255  # 빨간색 채널(BGR 중 R에 해당하는 2번 채널)만 마스크 적용

# 원본 이미지에 빨간색 덮어씌우기
overlayed_image = cv2.addWeighted(original_image, 1, red_overlay, 0.5, 0)

# 이미지 출력
plt.imshow(cv2.cvtColor(overlayed_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


In [ ]:
#태스트 데이터 opencv활용 contour 검출

plt.subplot(1, 2, 1)
plt.title("Contour Image")
plt.imshow(test_image_resized[0,:,:,0], cmap='gray')


plt.subplot(1, 2, 2)
plt.title("Contour Image")
REAL_contour_img = np.zeros_like(test_image_resized[0,:,:,0])
_, REAL_thresholded = cv2.threshold(test_image_resized[0,:,:,0], 150, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
REAL_contours_detected, _ = cv2.findContours(REAL_thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(REAL_contour_img, REAL_contours_detected, -1, (255, 255, 255), 1)
plt.imshow(REAL_contour_img, cmap='gray')

In [ ]:
# 이미지 불러오기
Area_image = predicted_contours[0, :, :, 0]


# 이진화된 이미지에서 하얀색 픽셀(값이 255인 픽셀)의 개수 세기
white_pixel_count = np.sum(Area_image == 255)

# 전체 픽셀 수 계산
total_pixel_count = Area_image.size

# 하얀색 면적 비율 계산
white_area_ratio = white_pixel_count / total_pixel_count

# 결과 출력
print(f"하얀색 픽셀의 개수: {white_pixel_count}")
print(f"전체 픽셀 수: {total_pixel_count}")
print(f"하얀색 면적 비율: {white_area_ratio * 100:.2f}%")

# 이미지 시각화
plt.imshow(Area_image, cmap='gray')
plt.title("Binary Image")
plt.show()
